In [1]:
# Importo librerías necesarias para la ejecución del proyecto
import pandas as pd

LIMIT = 10000000

In [2]:

def clean_columns(columns):
    try:
        columns_clean = []
        for column in columns:
            column = column.replace("-", "").upper()
            column = "_".join(column.split())
            columns_clean.append(column)
        return columns_clean
    except Exception as e:
        print(e)



In [3]:
# Dataset de transactiones
df_transactions = pd.read_csv("./../data/raw/credit_card_transactions-ibm_v2.csv")
df_transactions = df_transactions.tail(10000000)
df_transactions.head(2)


,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
14386900,1174,2,2019,2,2,16:08,$26.10,Chip Transaction,1913477460590765860,Los Angeles,CA,90002.0,5300,NaN,No
14386901,1174,2,2019,2,2,22:14,$64.56,Chip Transaction,4722913068560264812,Reno,NV,89511.0,5411,NaN,No


In [4]:
# Dataset de usuarios
df_users = pd.read_csv("./../data/raw/sd254_users.csv")
df_users.head(2)


,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards
0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,NaN,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5
1,Sasha Sadr,53,68,1966,12,Female,3606 Federal Boulevard,NaN,Little Neck,NY,11363,40.76,-73.74,$37891,$77254,$191349,701,5


In [5]:
# Dataset de tarjetas
df_cards = pd.read_csv("./../data/raw/sd254_cards.csv")
df_cards.head(2)


,User,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web
0,0,0,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,0,1,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No


## Data cleaning

#### Renombrar las columnas

In [6]:
# Add index and clean columns to user and cards data
df_users = df_users.reset_index().rename(columns={'index': 'ID_user',
                                                  'Person': 'user_full_name'})
df_users.columns = clean_columns(df_users.columns)

df_cards = df_cards.rename(columns={'CARD INDEX': 'ID_card',
                                    'User': 'ID_user'})
df_cards.columns = clean_columns(df_cards.columns)

# Rename columns of user and card ID's and clean columns on transactions data
df_transactions = df_transactions.reset_index().rename(
                                                columns={
                                                    'User': 'ID_user',
                                                    'Card': 'ID_card',
                                                    'index': 'ID_transaction'
                                                })
df_transactions.columns = clean_columns(df_transactions.columns)


### Control de valores nulos, formatos de las columnas y reporte de duplicados

#### Users

In [7]:
# Conteo de valores no nulos, con su respectivo tipo de dato por cada columna.
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID_USER                    2000 non-null   int64  
 1   USER_FULL_NAME             2000 non-null   object 
 2   CURRENT_AGE                2000 non-null   int64  
 3   RETIREMENT_AGE             2000 non-null   int64  
 4   BIRTH_YEAR                 2000 non-null   int64  
 5   BIRTH_MONTH                2000 non-null   int64  
 6   GENDER                     2000 non-null   object 
 7   ADDRESS                    2000 non-null   object 
 8   APARTMENT                  528 non-null    float64
 9   CITY                       2000 non-null   object 
 10  STATE                      2000 non-null   object 
 11  ZIPCODE                    2000 non-null   int64  
 12  LATITUDE                   2000 non-null   float64
 13  LONGITUDE                  2000 non-null   float

Con el resultado anterior, encontramos que debemos cambiar de tipo de dato las siguientes columnas:
* **\'Per Capita Income - Zipcode\'**: De object a float, ya que es un valor numérico precedido un signo $
* **:\'Yearly Income - Person\'**: De object a float, ya que es un valor numérico precedido un signo $
* **:\'Total Debt\'**: De object a float, ya que es un valor numérico precedido un signo $
* **:\'Zipcode\'**: De float a texto, ya que es un valor que no tiene un sentido ordinal.
* **:\'Apartment\'**: De float a object, ya que de acuerdo a la exploración, no cumple las condiciones para ser categorizado como numérico, por lo cual lo ideal es que fuera tratado como texto. Además, los valores nulos se deciden reemplazar con '0', con el fin de indicar que no aplica o no existen datos allí.

En este caso no se encuentran valores duplicados, por lo cual, como paso final se guarda el resultado como archivo parquet, debido a sus beneficios de eficiencia de lectura, escritura y almacenamiento de los datos

In [8]:
# Rows duplicated
df_users[df_users.duplicated()]

,ID_USER,USER_FULL_NAME,CURRENT_AGE,RETIREMENT_AGE,BIRTH_YEAR,BIRTH_MONTH,GENDER,ADDRESS,APARTMENT,CITY,STATE,ZIPCODE,LATITUDE,LONGITUDE,PER_CAPITA_INCOME_ZIPCODE,YEARLY_INCOME_PERSON,TOTAL_DEBT,FICO_SCORE,NUM_CREDIT_CARDS


In [9]:
# Limpieza y transformación de las columnas
df_users['PER_CAPITA_INCOME_ZIPCODE'] = df_users['PER_CAPITA_INCOME_ZIPCODE'].str[1:].astype(float)
df_users['YEARLY_INCOME_PERSON'] = df_users['YEARLY_INCOME_PERSON'].str[1:].astype(float)
df_users['TOTAL_DEBT'] = df_users['TOTAL_DEBT'].str[1:].astype(float)
df_users['ZIPCODE'] = df_users['ZIPCODE'].astype(str)
df_users['APARTMENT'] = df_users['APARTMENT'].fillna(0).astype(int).astype(str)

# Almacenamiento del dataframe final de usuarios
df_users.to_parquet("./../data/processed/data_users.parquet", engine='fastparquet')

In [10]:
print(f"Validación clave primaria: {df_users.ID_USER.nunique() == len(df_users)}")
print(f"Validación completitud de los datos: {df_users.isnull().sum().sum() == 0}")
print(f"Validación duplicidad de datos: {len(df_users[df_users.duplicated()]) == 0}")

Validación clave primaria: True
Validación completitud de los datos: True
Validación duplicidad de datos: True


#### Cards

In [11]:
# Conteo de valores no nulos, con su respectivo tipo de dato por cada columna.
df_cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID_USER                6146 non-null   int64 
 1   ID_CARD                6146 non-null   int64 
 2   CARD_BRAND             6146 non-null   object
 3   CARD_TYPE              6146 non-null   object
 4   CARD_NUMBER            6146 non-null   int64 
 5   EXPIRES                6146 non-null   object
 6   CVV                    6146 non-null   int64 
 7   HAS_CHIP               6146 non-null   object
 8   CARDS_ISSUED           6146 non-null   int64 
 9   CREDIT_LIMIT           6146 non-null   object
 10  ACCT_OPEN_DATE         6146 non-null   object
 11  YEAR_PIN_LAST_CHANGED  6146 non-null   int64 
 12  CARD_ON_DARK_WEB       6146 non-null   object
dtypes: int64(6), object(7)
memory usage: 624.3+ KB


Con el resultado anterior, encontramos que debemos cambiar de tipo de dato las siguientes columnas:
* **\'Credit Limit\'**: De object a float, ya que es un valor numérico precedido un signo $
* **\'Has Chip\'**: De object a bool, ya que es un atributo que tiene como posibles valores Si o No.
* **\'Card on Dark Web\'**: De object a bool, ya que es un atributo que tiene como posibles valores Si o No.
* **\'Card Number\'**: De int a texto, ya que es un valor que no tiene un sentido ordinal.
* **\'CVV\'**: De int a texto, ya que es un valor que no tiene un sentido ordinal.

En este caso no se encuentran valores duplicados, por lo cual, como paso final se guarda el resultado como archivo parquet, debido a sus beneficios de eficiencia de lectura, escritura y almacenamiento de los datos

In [12]:
# Columns to report as duplicates
df_cards[df_cards.duplicated()]

,ID_USER,ID_CARD,CARD_BRAND,CARD_TYPE,CARD_NUMBER,EXPIRES,CVV,HAS_CHIP,CARDS_ISSUED,CREDIT_LIMIT,ACCT_OPEN_DATE,YEAR_PIN_LAST_CHANGED,CARD_ON_DARK_WEB


In [13]:
# Data cleaning and transformation
df_cards['CREDIT_LIMIT'] = df_cards['CREDIT_LIMIT'].str[1:].astype(float)
df_cards['HAS_CHIP'] =  df_cards['HAS_CHIP'].replace({'YES': True, 'NO': False}).astype(bool)
df_cards['CARD_ON_DARK_WEB'] =  df_cards['CARD_ON_DARK_WEB'].replace({'YES': True, 'NO': False}).astype(bool)
df_cards[['CARD_NUMBER', 'CVV']] = df_cards[['CARD_NUMBER', 'CVV']].astype(str)

# Storage of credit card data
df_cards.to_parquet("./../data/processed/data_cards.parquet", engine='fastparquet')


In [14]:
print(f"Validación clave primaria: {df_cards.ID_CARD.nunique() == len(df_cards)}")
print(f"Validación completitud de los datos: {df_cards.isnull().sum().sum() == 0}")
print(f"Validación duplicidad de datos: {len(df_cards[df_cards.duplicated()]) == 0}")


Validación clave primaria: False
Validación completitud de los datos: True
Validación duplicidad de datos: True


#### Transactions

In [16]:
# Percentage of empty values in each column
df_transactions.isnull().sum() / df_transactions.shape[0] * 100

ID_TRANSACTION     0.00000
ID_USER            0.00000
ID_CARD            0.00000
YEAR               0.00000
MONTH              0.00000
DAY                0.00000
TIME               0.00000
AMOUNT             0.00000
USE_CHIP           0.00000
MERCHANT_NAME      0.00000
MERCHANT_CITY      0.00000
MERCHANT_STATE    10.39978
ZIP               11.01091
MCC                0.00000
ERRORS?           98.42521
IS_FRAUD?          0.00000
dtype: float64

In [17]:
# Correction of the formatting of each column
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 16 columns):
 #   Column          Dtype  
---  ------          -----  
 0   ID_TRANSACTION  int64  
 1   ID_USER         int64  
 2   ID_CARD         int64  
 3   YEAR            int64  
 4   MONTH           int64  
 5   DAY             int64  
 6   TIME            object 
 7   AMOUNT          object 
 8   USE_CHIP        object 
 9   MERCHANT_NAME   int64  
 10  MERCHANT_CITY   object 
 11  MERCHANT_STATE  object 
 12  ZIP             float64
 13  MCC             int64  
 14  ERRORS?         object 
 15  IS_FRAUD?       object 
dtypes: float64(1), int64(8), object(7)
memory usage: 1.2+ GB


Con el resultado anterior, encontramos que debemos cambiar de tipo de dato las siguientes columnas:
* **\'Amount\'**: De object a float, ya que es un valor numérico precedido un signo $
* **:\'Merchant Name\'**: De int a object, ya que es un atributo anonimizado y su naturaleza sería tipo texto.
* **:\'MCC\'**: De int a object, ya que es un valor que no tiene un sentido ordinal.
* **:\'Is Fraud?\'**: De object a bool, ya que es un atributo que tiene como posibles valores Si o No.
* **:\'Zip\'**: De float a texto, ya que es un valor que no tiene un sentido ordinal. En este atributo se encuentra que hay un alto porcentaje de valores vacíos, sin embargo, haciendo un análisis a esta columna, se encuentra que sólo tiene valores para las transacciones realizadas físicamente dentro de EEUU, por lo cual, se decide completar los valores con 'N/A', pues no tenemos mayor detalle del lugar de la realización de la compra. Es de anotar que también sería útil traer un codigo zip estandar por cada ciudad, para así poder utilizar herramientas BI y ver las locaciones de manera más detallada.
* **\'Merchant State\'**: Similar a la anterior columna Zip, existen valores nulos que están relacionados con las transacciones realizadas fuera de EEUU, por lo cual también se imputará con 'N/A'
* **:\'Errors?\'**: Tiene valores nulos, pero debido a su naturaleza es normal y se decide reemplazar por 'N/A'

Además, se ve la oportunidad de crear un campo de fecha, el cual concatene los campos de Year, Month y Day.

En este caso no se encuentran valores duplicados, por lo cual, como paso final se guarda el resultado como archivo parquet, debido a sus beneficios de eficiencia de lectura, escritura y almacenamiento de los datos


Finalmente, se guarda el resultado como archivo parquet, debido a sus beneficios de eficiencia de lectura, escritura y almacenamiento de los datos

In [18]:
# Columns to report as duplicates
df_transactions[df_transactions.duplicated()]

,ID_TRANSACTION,ID_USER,ID_CARD,YEAR,MONTH,DAY,TIME,AMOUNT,USE_CHIP,MERCHANT_NAME,MERCHANT_CITY,MERCHANT_STATE,ZIP,MCC,ERRORS?,IS_FRAUD?


In [ ]:
# Data cleaning and transformation
df_transactions['AMOUNT'] = df_transactions['AMOUNT'].str[1:].astype(float)
df_transactions['MERCHANT_NAME'] = df_transactions['MERCHANT_NAME'].astype(str)
df_transactions['MCC'] = df_transactions['MCC'].astype(str)
df_transactions['IS_FRAUD?'] =  df_transactions['IS_FRAUD?'].replace({'Yes': True, 'No': False}).astype(bool)
df_transactions['ZIP'] = df_transactions['ZIP'].fillna(0).astype(int).astype(str).replace({'0': 'N/A'})
df_transactions['MERCHANT_STATE'] = df_transactions.fillna('N/A')['MERCHANT_STATE']
df_transactions['ERRORS?'] = df_transactions['ERRORS?'].fillna('N/A')
df_transactions['TRANSACTION_DATE'] = pd.to_datetime(df_transactions[['YEAR', 'MONTH', 'DAY']])

# Removing duplicate data
df_transactions.drop_duplicates(inplace=True)

# Storage of transaction data
df_transactions.to_parquet("./../data/processed/data_transactions.parquet", engine='fastparquet')
